In [15]:
import pickle
from utils.analysis_from_interaction import *
from egg.core.language_analysis import Disent
from language_analysis_local import TopographicSimilarityConceptLevel, encode_target_concepts_for_topsim

# calculate metrics from stored interactions

In [16]:
datasets = ('(3,4)', '(3,8)', '(3,16)', '(4,4)', '(4,8)', '(5,4)')
n_attributes = (3, 3, 3, 4, 4, 5)
n_values = (4, 8, 16, 4, 8, 4)
n_epochs = 300
#paths = ['results/' + d + '_sample_scaling_10_balanced_False_vsf_3/' for d in datasets]
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]



In [3]:
datasets = ('(4,4)',)
n_attributes = (4,)
n_values = (4,)
n_epochs = 100
#paths = ['results/' + d + '_sample_scaling_10_balanced_False_vsf_3/' for d in datasets]
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]

In [17]:
context_unaware = False # whether original or context_unaware simulations are evaluated
length_cost = False # whether length_cost was applied
no_cost = True # set to True to consider baselines without a cost
if length_cost:
    if context_unaware and not no_cost:
        setting = 'length_cost/context_unaware'
    elif context_unaware and no_cost:
        setting = 'length_cost/no_cost_context_unaware'
    elif not context_unaware and not no_cost:
        setting = 'length_cost/context_aware'
    else:
        setting = 'length_cost/no_cost_context_aware'
else:
    if context_unaware:
        setting = 'context_unaware'
    else:
        setting = 'standard'

## entropy scores: MI, effectiveness, efficiency

In [5]:
for d in range(len(datasets)):
    
    for run in range(5):

        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/' 
        print(path_to_run)
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = information_scores(interaction, attributes, values, normalizer="arithmetic")

        pickle.dump(scores, open(path_to_run + 'entropy_scores.pkl', 'wb'))
    

results/(4,4)_game_size_10_vsf_3/length_cost/no_cost_context_unaware/0/


/Users/kkobrock/Projects/phdproject1/emergent-abstractions/utils/analysis_from_interaction.py:310: RuntimeWarning: invalid value encountered in divide
  normalized_MI_conc_x_cont = ((m_entropy_concept_x_context + c_entropy_concept_x_context - joint_entropy_concept_x_context)
/Users/kkobrock/Projects/phdproject1/emergent-abstractions/utils/analysis_from_interaction.py:319: RuntimeWarning: invalid value encountered in divide
  normalized_effectiveness_conc_x_cont = ((joint_entropy_concept_x_context - m_entropy_concept_x_context)
/Users/kkobrock/Projects/phdproject1/emergent-abstractions/utils/analysis_from_interaction.py:326: RuntimeWarning: invalid value encountered in divide
  normalized_consistency_conc_x_cont = (joint_entropy_concept_x_context - c_entropy_concept_x_context) / m_entropy_concept_x_context


results/(4,4)_game_size_10_vsf_3/length_cost/no_cost_context_unaware/1/
results/(4,4)_game_size_10_vsf_3/length_cost/no_cost_context_unaware/2/
results/(4,4)_game_size_10_vsf_3/length_cost/no_cost_context_unaware/3/
results/(4,4)_game_size_10_vsf_3/length_cost/no_cost_context_unaware/4/


##  message length

In [57]:
# we evaluated message length per hierarchy level after training but 
# you can also use the HierarchicalMessageLength callback and store the results 
# TODO: Message length results look weird, needs to be fixed!

for d in range(len(datasets)):
    
    for run in range(5): 
        
        path_to_run = paths[d] + '/' + str(setting) +'/' + str(run) + '/'
        path_to_interaction = (path_to_run + 'interactions/train/interactions_epoch' + str(n_epochs))
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = message_length_per_hierarchy_level(interaction, attributes)
        
        pickle.dump(scores, open(path_to_run + 'message_length_hierarchical.pkl', 'wb'))

tensor([[14,  8,  8,  ..., 15,  1,  0],
        [ 5, 15,  5,  ..., 14, 14,  0],
        [ 4,  3,  2,  ...,  7,  2,  0],
        ...,
        [ 5,  5, 15,  ...,  9, 10,  0],
        [12,  2,  3,  ...,  9, 10,  0],
        [14,  8,  8,  ...,  2,  2,  0]])
tensor([1, 1, 1,  ..., 1, 1, 1])
message length tensor([20, 20, 20,  ..., 20, 20, 20])
tensor([[14,  8,  8,  ..., 15,  1,  0],
        [ 5, 15,  5,  ..., 14, 14,  0],
        [ 4,  3,  2,  ...,  7,  2,  0],
        ...,
        [ 5,  5, 15,  ...,  9, 10,  0],
        [12,  2,  3,  ...,  9, 10,  0],
        [14,  8,  8,  ...,  2,  2,  0]])
tensor([1, 1, 1,  ..., 1, 1, 1])
hierarchical [20.0, 20.0, 20.0, 20.0]
tensor([[ 2,  2, 13,  ...,  2, 11,  0],
        [ 9,  9,  2,  ...,  2,  8,  0],
        [ 2,  2,  2,  ...,  2,  2,  0],
        ...,
        [ 2,  3,  2,  ...,  2,  8,  0],
        [ 2, 15,  2,  ...,  7, 12,  0],
        [10,  2,  2,  ...,  1,  2,  0]])
tensor([1, 1, 1,  ..., 1, 1, 1])
message length tensor([20, 20, 20,  ..., 20, 20

##  symbol redundancy

In [58]:
for d in range(len(datasets)):
    
    attributes = n_attributes[d]
    values = n_values[d]
    vs_factor = int(paths[d][-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1
    
    for run in range(5): 
        
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        #symbol_f = np.load(path_to_run + 'symbols_pernsame.npy')
        #path_to_interaction = (path_to_run + 'interactions/train/interactions_epoch' + str(n_epochs))
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)
        redundancy, MI = symbol_frequency(interaction, attributes, values, vocab_size)
        
        scores = {'symbol_redundancy': redundancy, 'MI_symbol-attribute_value': MI}
        
        pickle.dump(scores, open(path_to_run + 'symbol_redundancy.pkl', 'wb'))

/Users/kkobrock/Projects/phdproject1/emergent-abstractions/utils/analysis_from_interaction.py:443: RuntimeWarning: invalid value encountered in divide
  return symbol_frequency / att_val_frequency, mutual_information
/Users/kkobrock/Projects/phdproject1/emergent-abstractions/utils/analysis_from_interaction.py:443: RuntimeWarning: invalid value encountered in divide
  return symbol_frequency / att_val_frequency, mutual_information
/Users/kkobrock/Projects/phdproject1/emergent-abstractions/utils/analysis_from_interaction.py:443: RuntimeWarning: invalid value encountered in divide
  return symbol_frequency / att_val_frequency, mutual_information
/Users/kkobrock/Projects/phdproject1/emergent-abstractions/utils/analysis_from_interaction.py:443: RuntimeWarning: invalid value encountered in divide
  return symbol_frequency / att_val_frequency, mutual_information
/Users/kkobrock/Projects/phdproject1/emergent-abstractions/utils/analysis_from_interaction.py:443: RuntimeWarning: invalid value enc

##  compositionality scores: topsim, posdis, bosdis

### topsim

In [8]:
# topsim
# although topsim values are stored throughout training if callbacks are verbose, we reevaluate the
# final topsim scores with more data points 

samples = 5000 # maybe shuffle from these because otherwise I just take the first 5,000 (which might not be the best)
for d, dataset in enumerate(datasets):
    
    dim = [n_values[d]]*n_attributes[d]
    
    for run in range(5):
        print("dataset", dataset, "run", run)
        
        topsim_final = {}
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        
        TOPSIM = TopographicSimilarityConceptLevel(dim, is_gumbel=True)
        
        for mode in ['train', 'val']:

            if mode == 'train':
                interaction = torch.load(path_to_interaction_train)
            elif mode == 'val':
                interaction = torch.load(path_to_interaction_val)
                
                  
            messages = interaction.message.argmax(dim=-1)
            sender_input = interaction.sender_input
            n_targets = int(sender_input.shape[1]/2)
            # get target objects and fixed vectors to re-construct concepts
            target_objects = sender_input[:, :n_targets]
            target_objects = k_hot_to_attributes(target_objects, n_values[d])
            # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
            (objects, fixed) = retrieve_concepts_sampling(target_objects)
            # add one such that zero becomes an empty attribute for the calculation (_)
            objects = objects + 1
            concepts = torch.from_numpy(objects * (np.array(fixed)))
            specific_idx = np.where(np.sum(fixed, axis=1)==n_attributes[d])[0]
            messages_specific = messages[specific_idx]
            concepts_specific = concepts[specific_idx]
            
            generic_idx = np.where(np.sum(fixed, axis=1)==1)[0]
            messages_generic = messages[generic_idx]
            concepts_generic = concepts[generic_idx]

            messages = [msg.tolist() for msg in messages]
            messages_specific = [msg.tolist() for msg in messages_specific]
            messages_generic = [msg.tolist() for msg in messages_generic]

            encoded_input = encode_target_concepts_for_topsim(sender_input)
            # randomly take samples when more than 5000 samples are available
            # if len(encoded_input) > samples: 
            #     print("sampling")
            #     sample_indices = random.sample(range(len(encoded_input)), samples)
            #     sampled_input = [encoded_input[i] for i in sample_indices]
            #     sampled_messages = [messages[i] for i in sample_indices]
            #     print("start computing")
            #     print(len(sampled_input), len(sampled_input[0]), len(sampled_input[0][0]))
            #     topsim = TOPSIM.compute_topsim(sampled_input, sampled_messages)
            # else:
            topsim = TOPSIM.compute_topsim(encoded_input[0:samples], messages[0:samples]) # default: hausdorff distance for concepts, edit distance for messages
            # if len(concepts_specific) > samples:
            #     print("sampling specific")
            #     sample_indices_specific = random.sample(range(len(concepts_specific)), samples)
            #     sampled_input_specific = [concepts_specific[i] for i in sample_indices_specific]
            #     sampled_messages_specific = [messages_specific[i] for i in sample_indices_specific]
            #     topsim_specific = TOPSIM.compute_topsim(sampled_input_specific, sampled_messages_specific, 
            #                                             meaning_distance_fn="edit")
            # else:
            topsim_specific = TOPSIM.compute_topsim(concepts_specific[0:samples], messages_specific[0:samples], 
                                                        meaning_distance_fn="edit")
            
            topsim_generic = TOPSIM.compute_topsim(concepts_generic[0:samples], messages_generic[0:samples],
                                                   meaning_distance_fn="edit")

            print('... topsim computed')

            topsim_final['topsim_' + mode] = topsim
            topsim_final['topsim_specific_' + mode] = topsim_specific
            topsim_final['topsim_generic_' + mode] = topsim_generic
    
        pickle.dump(topsim_final, open(path_to_run +  "topsim_final.pkl", "wb" ) )
        print(topsim_final)        

dataset (4,4) run 0
... topsim computed
... topsim computed
{'topsim_train': 0.4961544843847002, 'topsim_specific_train': 0.5613587001371683, 'topsim_generic_train': 0.356023115741368, 'topsim_val': 0.5183962773875957, 'topsim_specific_val': 0.6014905644104569, 'topsim_generic_val': 0.34987064972295523}
dataset (4,4) run 1
... topsim computed
... topsim computed
{'topsim_train': 0.4911624839020278, 'topsim_specific_train': 0.5448991474589403, 'topsim_generic_train': 0.1824133804995301, 'topsim_val': 0.5042482234534768, 'topsim_specific_val': 0.5484302690765211, 'topsim_generic_val': 0.20498589414276477}
dataset (4,4) run 2
... topsim computed
... topsim computed
{'topsim_train': 0.4522702414218579, 'topsim_specific_train': 0.4932977631876992, 'topsim_generic_train': 0.2367133421060698, 'topsim_val': 0.44841049933297644, 'topsim_specific_val': 0.49703150963945175, 'topsim_generic_val': 0.2573979986295118}
dataset (4,4) run 3
... topsim computed
... topsim computed
{'topsim_train': 0.317

#### Topsim over time

In [ ]:
for d, dataset in enumerate(datasets):
    
    for run in range(5):
        print("dataset", dataset, "run", run)
        
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        
        for mode in ['train', 'val']:

            if mode == 'train':
                interaction = torch.load(path_to_interaction_train)
            elif mode == 'val':
                interaction = torch.load(path_to_interaction_val)

        messages = interaction.message.argmax(dim=-1)
        sender_input = interaction.sender_input
        messages = [msg.tolist() for msg in messages]
        encoded_input = encode_target_concepts_for_topsim(sender_input)
        dim = [n_values[0]] * n_attributes[0]
        TOPSIM = TopographicSimilarityConceptLevel(dim, is_gumbel=True)
        
        samples = 5000
        num_batches = len(messages) // samples + (len(messages) % samples > 0)
        topsim_over_time = []
        
        for i in range(num_batches):
            messages_batch = messages[i * samples:(i + 1) * samples]
            topsim = TOPSIM.compute_topsim(encoded_input[i * samples:(i + 1) * samples], messages_batch)
            topsim_over_time.append(topsim)
            
        pickle.dump(topsim_over_time, open(path_to_run +  "topsim_over_time.pkl", "wb" ) )

dataset (3,4) run 0
dataset (3,4) run 1
dataset (3,4) run 2
dataset (3,4) run 3
dataset (3,4) run 4
dataset (3,8) run 0
dataset (3,8) run 1
dataset (3,8) run 2
dataset (3,8) run 3
dataset (3,8) run 4
dataset (3,16) run 0
dataset (3,16) run 1


### Posdis and Bosdis

In [59]:
# use Disent callback from egg

for d in range(len(datasets)): 
    
    path = paths[d]
    dim = [n_values[d]] * n_attributes[d]
    n_features = n_attributes[d] * n_values[d]
    vs_factor = int(path[-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1
    
    print("data set", dim)
    
    for run in range(5):
        
        posdis_bosdis = {}
    
        path_to_run = paths[d] + '/' + str(setting) +'/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)
        
        messages = interaction.message.argmax(dim=-1)
        sender_input = interaction.sender_input
        n_targets = int(sender_input.shape[1]/2)
        # get target objects and fixed vectors to re-construct concepts
        target_objects = sender_input[:, :n_targets]
        target_objects = k_hot_to_attributes(target_objects, n_values[d])
        # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
        (objects, fixed) = retrieve_concepts_sampling(target_objects)
        # add one such that zero becomes an empty attribute for the calculation (_)
        objects = objects + 1
        concepts = torch.from_numpy(objects * (np.array(fixed)))

        # concrete/specific concepts: where all attributes are fixed
        concepts_specific = torch.tensor(
            objects[torch.sum(torch.from_numpy(fixed), dim=1) == n_attributes[d]])
        messages_specific = messages[torch.sum(torch.from_numpy(fixed), dim=1) == n_attributes[d]]

        # generic concepts: where only one attribute is fixed
        concepts_generic = torch.tensor(
            objects[torch.sum(torch.from_numpy(fixed), dim=1) == 1])
        messages_generic = messages[torch.sum(torch.from_numpy(fixed), dim=1) == 1]
        
        posdis_specific = Disent.posdis(concepts_specific, messages_specific)
        bosdis_specific = Disent.bosdis(concepts_specific, messages_specific, vocab_size)

        posdis_generic = Disent.posdis(concepts_generic, messages_generic)
        bosdis_generic = Disent.bosdis(concepts_generic, messages_generic, vocab_size)
        
        posdis = Disent.posdis(torch.from_numpy(objects), messages)
        bosdis = Disent.bosdis(torch.from_numpy(objects), messages, vocab_size)
        
        posdis_bosdis['posdis_specific'] = posdis_specific
        posdis_bosdis['bosdis_specific'] = bosdis_specific
        posdis_bosdis['posdis_generic'] = posdis_generic
        posdis_bosdis['bosdis_generic'] = bosdis_generic
        posdis_bosdis['posdis'] = posdis
        posdis_bosdis['bosdis'] = bosdis

        print(posdis_bosdis)
    
        pickle.dump(posdis_bosdis, open(path_to_run + "posdis_bosdis.pkl", "wb" ) )

data set [4, 4, 4, 4]
{'posdis_specific': 0.0465884767472744, 'bosdis_specific': 0.2928831875324249, 'posdis_generic': 0.03174097090959549, 'bosdis_generic': 0.06935141235589981, 'posdis': 0.0335584357380867, 'bosdis': 0.24937672913074493}
{'posdis_specific': 0.07036396116018295, 'bosdis_specific': 0.2058870941400528, 'posdis_generic': 0.022420044988393784, 'bosdis_generic': 0.048999253660440445, 'posdis': 0.0441184863448143, 'bosdis': 0.19311127066612244}
{'posdis_specific': 0.014630235731601715, 'bosdis_specific': 0.17614348232746124, 'posdis_generic': 0.043402787297964096, 'bosdis_generic': 0.05605326592922211, 'posdis': 0.014388194307684898, 'bosdis': 0.15945889055728912}
{'posdis_specific': 0.09702602028846741, 'bosdis_specific': 0.2543882429599762, 'posdis_generic': 0.07296779751777649, 'bosdis_generic': 0.08056525141000748, 'posdis': 0.09423286467790604, 'bosdis': 0.216058149933815}
{'posdis_specific': 0.052317507565021515, 'bosdis_specific': 0.35632622241973877, 'posdis_generic

#### Posdis and bosdis concept x context

In [65]:
# bosdis concept x context
for d in range(len(datasets)):

    vs_factor = int(paths[d][-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1
    
    for run in range(5):

        path_to_run = paths[d] + '/' + str(setting) +'/' + str(run) + '/' 
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = bosdis(interaction, attributes, values, vocab_size)

        pickle.dump(scores, open(path_to_run + 'bosdis_scores.pkl', 'wb'))

In [66]:
# posdis concept x context
for d in range(len(datasets)):

    vs_factor = int(paths[d][-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1

    for run in range(5):
        path_to_run = paths[d] + '/' + str(setting) + '/' + str(run) + '/'
        print(path_to_run)
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        #print(path_to_interaction)
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = posdis(interaction, attributes, values, vocab_size)

        pickle.dump(scores, open(path_to_run + 'posdis_scores.pkl', 'wb'))

results/(4,4)_game_size_10_vsf_3//length_cost/no_cost_context_unaware/0/
results/(4,4)_game_size_10_vsf_3//length_cost/no_cost_context_unaware/1/
results/(4,4)_game_size_10_vsf_3//length_cost/no_cost_context_unaware/2/
results/(4,4)_game_size_10_vsf_3//length_cost/no_cost_context_unaware/3/
results/(4,4)_game_size_10_vsf_3//length_cost/no_cost_context_unaware/4/


## co-occurrences

In [ ]:
# Not yet implemented:

for d in range(len(datasets)):
    
    vs_factor = int(paths[d][-2])
    
    for run in range(5): 
        
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        
        scores = cooccurrence_per_hierarchy_level(interaction, attributes, values, vs_factor)

        print(scores)
        
        pickle.dump(scores, open(path_to_run + 'normalized_cooccurrence.pkl', 'wb'))
    